# Create monthly CESM atmospheric deposition files

NCAR CESM output:
Community Earth System Model (Community Atmosphere Model - CAM https://agupubs.onlinelibrary.wiley.com/doi/abs/10.1002/2013MS000279) output: https://www.earthsystemgrid.org/; CESM1 CAM5 BGC Large Ensemble Atmosphere Post Processed Data, Monthly Averages.

Specific run output: https://www.earthsystemgrid.org/dataset/ucar.cgd.ccsm4.CESM_CAM5_BGC_LE.atm.proc.monthly_ave.html

In [1]:
import matplotlib.pyplot as plt
import netCDF4 as nc
import numpy as np
import datetime
from mpl_toolkits.basemap import Basemap, cm
import cmocean
import matplotlib
import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

In [4]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [2]:
print('Run id: ucar.cgd.ccsm4.CESM_CAM5_BGC_LE')

Run id: ucar.cgd.ccsm4.CESM_CAM5_BGC_LE


In [3]:
mask  = nc.Dataset('/ocean/brogalla/GEOTRACES/ariane_runs/ANHA12_Ariane_mesh.nc')
tmask = np.array(mask.variables['tmask'])
tmask = np.array(tmask[0,:,:,:])

mesh  = nc.Dataset('/data/brogalla/old/meshmasks/ANHA12_mesh1.nc')
mlons = np.array(mesh.variables['nav_lon'])
mlats = np.array(mesh.variables['nav_lat'])
mlons = np.array(mlons)
mlats = np.array(mlats)

cond = (tmask > 0.1) 
Z_masked = np.ma.masked_where(cond, tmask) 

In [4]:
def interp_np(nav_lon, nav_lat, var_in, lon_ANHA12, lat_ANHA12):
    ''' Interpolate some field to ANHA12 grid.
        The function is based on the bilinear interpolation in scipy, griddata 
        =======================================================================
            nav_lon, nav_lat        : input field lon/lat
            lon_ANHA12, lat_ANHA12  : ANHA12 defined lons/lats
            var_in                  : 2-D model variable
    '''
    from scipy.interpolate import griddata
    LatLonPair = (nav_lon.flatten(), nav_lat.flatten())
    a = nav_lon.flatten()
    b = nav_lat.flatten()
    c = var_in.flatten()
    print(a.shape, b.shape, c.shape, lon_ANHA12.shape, lat_ANHA12.shape)
    var_out = griddata(LatLonPair, var_in.flatten(), (lon_ANHA12, lat_ANHA12), method='linear')
    # Take nearest neighbour interpolation to fill nans
    var_fill = griddata(LatLonPair, var_in.flatten(), (lon_ANHA12, lat_ANHA12), method='nearest')
    
    var_out[np.isnan(var_out)] = var_fill[np.isnan(var_out)]
    return var_out

In [5]:
def save_file(filename, field1, field2, field3):
    ncd = nc.Dataset(filename, 'w', zlib=True)
#     nc_tools.init_dataset_attrs(
#         ncd,
#         title='dust input field',
#         notebook_name='',
#         nc_filepath='./monthly_CESM_forcing.nc',
#         comment='CESM ucar.cgd.ccsm4.CESM_CAM5_BGC_LE dust deposition field')

    ncd.createDimension('x',len(mesh.dimensions['x']))
    ncd.createDimension('y',len(mesh.dimensions['y']))
    ncd.createDimension('time_counter',None)
    
    # variables
    fine_dust             = ncd.createVariable('fdust', 'float64', ('y','x'))
    fine_dust.units       = 'g/m2/s'
    fine_dust.long_name   = 'Fine dust deposition flux'  
    fine_dust.coordinates = 'nav_lon nav_lat'
    fine_dust[:]          = field1
    
    coarse_dust             = ncd.createVariable('cdust', 'float64', ('y','x'))
    coarse_dust.units       = 'g/m2/s'
    coarse_dust.long_name   = 'Coarse dust deposition flux'  
    coarse_dust.coordinates = 'nav_lon nav_lat'
    coarse_dust[:]          = field2
    
    black_c             = ncd.createVariable('bc', 'float64', ('y','x'))
    black_c.units       = 'g/m2/s'
    black_c.long_name   = 'Black carbon deposition flux'  
    black_c.coordinates = 'nav_lon nav_lat'
    black_c[:]          = field3
    
    print('saved ', filename)

    ncd.close()
    return

For Mode MAM3 (https://www.geosci-model-dev.net/5/709/2012/):

- a1 --- Aitken mode --- 0.015-0.053 μm
- a2 --- Accumulation mode --- 0.058-0.27 μm
- a3 --- Coarse mode --- 0.80-3.65 μm

Indicate year for which to create files:

In [139]:
file_year = 2020

Load data:

In [140]:
folder = '/ocean/brogalla/GEOTRACES/data/NCAR/'
dst_a1DDF_f   = 'merged_dst_a1DDF.nc'
dst_a1SFWET_f = 'merged_dst_a1SFWET.nc'
dst_a3DDF_f   = 'merged_dst_a3DDF.nc'
dst_a3SFWET_f = 'merged_dst_a3SFWET.nc'
bc_a1DDF_f    = 'merged_bc_a1DDF.nc'
bc_a1SFWET_f  = 'merged_bc_a1SFWET.nc'

dst_a1D = nc.Dataset(folder+dst_a1DDF_f)
dst_a1S = nc.Dataset(folder+dst_a1SFWET_f)
dst_a3D = nc.Dataset(folder+dst_a3DDF_f)
dst_a3S = nc.Dataset(folder+dst_a3SFWET_f)
bc_a1D  = nc.Dataset(folder+bc_a1DDF_f)
bc_a1S  = nc.Dataset(folder+bc_a1SFWET_f)

In [141]:
dst_a1DDF   = dst_a1D.variables['dst_a1DDF']
dst_a1SFWET = dst_a1S.variables['dst_a1SFWET']
dst_a3DDF   = dst_a3D.variables['dst_a3DDF']
dst_a3SFWET = dst_a3S.variables['dst_a3SFWET']
BC_a1DDF    = bc_a1D.variables['bc_a1DDF']
BC_a1SFWET  = bc_a1S.variables['bc_a1SFWET']

In [142]:
lon  = dst_a1D.variables['lon']
lat  = dst_a1D.variables['lat']
date = dst_a1D.variables['date']

In [143]:
lon = np.array(lon)
for i in range(0,len(lon)):
    if lon[i] >= 180:
        lon[i] = -360+lon[i]

In [144]:
# change lons and lats array dimensions: (192x288)
lons, lats = np.meshgrid(lon,lat)
lons=np.array(lons)
lats=np.array(lats)

identify array indices corresponding to start and end of file_year:

In [145]:
file_date_start = file_year*10000 + 101
file_date_end = file_year*10000 + 1201

In [146]:
start_index = []
end_index = []

for i in range(0,len(date)):
    if date[i] == file_date_start:
        start_index = i
    elif date[i] == file_date_end:
        end_index = i
        
print('start index: ', start_index)
print('end index: ', end_index)

start index:  2039
end index:  2050


In [147]:
print(date[start_index], date[end_index])

20200101 20201201


Sum wet and dry deposition fields & zero any negative deposition fluxes:

In [148]:
dst_a1DDF = np.array(dst_a1DDF)
dst_a1SFWET = np.array(dst_a1SFWET)
dst_a3DDF = np.array(dst_a3DDF)
dst_a3SFWET = np.array(dst_a3SFWET)
BC_a1DDF = np.array(BC_a1DDF)
BC_a1SFWET = np.array(BC_a1SFWET)

In [149]:
# zero negative deposition fluxes:
dst_a1DDF[dst_a1DDF < 0] = 0
dst_a1SFWET[dst_a1SFWET < 0] = 0
dst_a3DDF[dst_a3DDF < 0] = 0
dst_a3SFWET[dst_a3SFWET < 0] = 0
BC_a1DDF[BC_a1DDF < 0] = 0
BC_a1SFWET[BC_a1SFWET < 0] = 0

Convert kg/m2/s to g/m2/s

In [150]:
dst_a1 = np.add(dst_a1DDF[start_index:end_index+1,:,:]*10**3, dst_a1SFWET[start_index:end_index+1, :,:]*10**3)
dst_a3 = np.add(dst_a3DDF[start_index:end_index+1,:,:]*10**3, dst_a3SFWET[start_index:end_index+1, :,:]*10**3)
bc_a1 = np.add(BC_a1DDF[start_index:end_index+1,:,:]*10**3, BC_a1SFWET[start_index:end_index+1, :,:]*10**3)

In [151]:
print('Check that output is of the correct dimensions: ', dst_a1.shape, dst_a3.shape, bc_a1.shape)

Check that output is of the correct dimensions:  (12, 192, 288) (12, 192, 288) (12, 192, 288)


In [152]:
dst_a1_flat = np.reshape(dst_a1, (12, 55296))
dst_a3_flat = np.reshape(dst_a3, (12, 55296))
bc_a1_flat = np.reshape(bc_a1, (12, 55296))

In [153]:
print('dust a1:')
print(np.max(np.nanmax(dst_a1_flat, axis = 1)))
print(np.min(np.nanmin(dst_a1_flat, axis = 1)))
print('dust a3:')
print(np.max(np.nanmax(dst_a3_flat, axis = 1)))
print(np.min(np.nanmin(dst_a3_flat, axis = 1)))
print('bc a1:')
print(np.max(np.nanmax(bc_a1_flat, axis = 1)))
print(np.min(np.nanmin(bc_a1_flat, axis = 1)))

dust a1:
6.392681e-07
5.5163804e-16
dust a3:
0.00016161492
2.5886011e-14
bc a1:
6.8735084e-09
9.667789e-17


Interpolate to ANHA12 grid:

In [154]:
dst_a1 = np.array(dst_a1)
dst_a3 = np.array(dst_a3)
bc_a1 = np.array(bc_a1)

In [155]:
interp_dst_a1 = np.empty((12, 2400, 1632))
interp_dst_a3 = np.empty((12, 2400, 1632))
interp_bc_a1  = np.empty((12, 2400, 1632))

# loop over the months:
for i in range(0,12):
    interp_dst_a1[i,:,:] = interp_np(lons, lats, dst_a1[i,:,:], mlons, mlats)
    interp_dst_a3[i,:,:] = interp_np(lons, lats, dst_a3[i,:,:], mlons, mlats)
    interp_bc_a1[i,:,:]  = interp_np(lons, lats, bc_a1[i,:,:], mlons, mlats)

(55296,) (55296,) (55296,) (2400, 1632) (2400, 1632)
(55296,) (55296,) (55296,) (2400, 1632) (2400, 1632)
(55296,) (55296,) (55296,) (2400, 1632) (2400, 1632)
(55296,) (55296,) (55296,) (2400, 1632) (2400, 1632)
(55296,) (55296,) (55296,) (2400, 1632) (2400, 1632)
(55296,) (55296,) (55296,) (2400, 1632) (2400, 1632)
(55296,) (55296,) (55296,) (2400, 1632) (2400, 1632)
(55296,) (55296,) (55296,) (2400, 1632) (2400, 1632)
(55296,) (55296,) (55296,) (2400, 1632) (2400, 1632)
(55296,) (55296,) (55296,) (2400, 1632) (2400, 1632)
(55296,) (55296,) (55296,) (2400, 1632) (2400, 1632)
(55296,) (55296,) (55296,) (2400, 1632) (2400, 1632)
(55296,) (55296,) (55296,) (2400, 1632) (2400, 1632)
(55296,) (55296,) (55296,) (2400, 1632) (2400, 1632)
(55296,) (55296,) (55296,) (2400, 1632) (2400, 1632)
(55296,) (55296,) (55296,) (2400, 1632) (2400, 1632)
(55296,) (55296,) (55296,) (2400, 1632) (2400, 1632)
(55296,) (55296,) (55296,) (2400, 1632) (2400, 1632)
(55296,) (55296,) (55296,) (2400, 1632) (2400,

### Write to NetCDF files:

In [156]:
location = '/data/brogalla/atmospheric_forcing/'

In [157]:
for i in range(1,13):
    if i < 10:
        save_file(location+'atm_flux_y'+str(file_year)+'m0'+str(i)+'.nc',interp_dst_a1[i-1,:,:],\
                  interp_dst_a3[i-1,:,:], interp_bc_a1[i-1,:,:])
    else:
        save_file(location+'atm_flux_y'+str(file_year)+'m'+str(i)+'.nc',interp_dst_a1[i-1,:,:],\
                  interp_dst_a3[i-1,:,:], interp_bc_a1[i-1,:,:])

saved  /data/brogalla/atmospheric_forcing/atm_flux_y2020m01.nc
saved  /data/brogalla/atmospheric_forcing/atm_flux_y2020m02.nc
saved  /data/brogalla/atmospheric_forcing/atm_flux_y2020m03.nc
saved  /data/brogalla/atmospheric_forcing/atm_flux_y2020m04.nc
saved  /data/brogalla/atmospheric_forcing/atm_flux_y2020m05.nc
saved  /data/brogalla/atmospheric_forcing/atm_flux_y2020m06.nc
saved  /data/brogalla/atmospheric_forcing/atm_flux_y2020m07.nc
saved  /data/brogalla/atmospheric_forcing/atm_flux_y2020m08.nc
saved  /data/brogalla/atmospheric_forcing/atm_flux_y2020m09.nc
saved  /data/brogalla/atmospheric_forcing/atm_flux_y2020m10.nc
saved  /data/brogalla/atmospheric_forcing/atm_flux_y2020m11.nc
saved  /data/brogalla/atmospheric_forcing/atm_flux_y2020m12.nc
